In [1]:
import setup
setup.init_django()

In [2]:
from market import services as market_services
from market import tasks as market_tasks
from market.models import Company

In [3]:
import json
from decouple import config

In [4]:
ticker = "AAPL"
name = "Apple"
company, _ = Company.objects.get_or_create(name=name, ticker=ticker)
company.id

2

In [5]:
market_tasks.sync_historical_stock_data(years_ago=1, company_ids=[company.id], use_celery=False, verbose=True)

# use celery / async
# market_tasks.sync_historical_stock_data.delay(years_ago=5, company_ids=[company.id], use_celery=True, verbose=False)

Historical sync days ago 30
dataset length 3398
Doing chunk 0
finished chunk 0
Doing chunk 1000
finished chunk 1000
Doing chunk 2000
finished chunk 2000
Doing chunk 3000
finished chunk 3000
30 done

Historical sync days ago 60
dataset length 6730
Doing chunk 0
finished chunk 0
Doing chunk 1000
finished chunk 1000
Doing chunk 2000
finished chunk 2000
Doing chunk 3000
finished chunk 3000
Doing chunk 4000
finished chunk 4000
Doing chunk 5000
finished chunk 5000
Doing chunk 6000
finished chunk 6000
60 done

Historical sync days ago 90
dataset length 10788
Doing chunk 0
finished chunk 0
Doing chunk 1000
finished chunk 1000
Doing chunk 2000
finished chunk 2000
Doing chunk 3000
finished chunk 3000
Doing chunk 4000
finished chunk 4000
Doing chunk 5000
finished chunk 5000
Doing chunk 6000
finished chunk 6000
Doing chunk 7000
finished chunk 7000
Doing chunk 8000
finished chunk 8000
Doing chunk 9000
finished chunk 9000
Doing chunk 10000
finished chunk 10000
90 done

Historical sync days ago 120
d

In [6]:
results = market_services.get_stock_indicators(ticker=ticker, days=90)
results

{'score': -1,
 'ticker': 'AAPL',
 'indicators': {'ma_5': 235.318,
  'ma_20': 245.604,
  'current_price': 233.78,
  'conservative_target': 248.1088,
  'aggressive_target': 256.9612,
  'average_price': 238.0477,
  'avg_volume': 2879.7123287671234,
  'latest_volume': 603,
  'volume_change_percent': -79.06040843120745,
  'rsi': 47.4629,
  'avg_gain': 0.9259,
  'avg_loss': 1.0249,
  'period': 14,
  'days': 90}}

In [7]:
results_as_json = json.dumps(results)
results_as_json

'{"score": -1, "ticker": "AAPL", "indicators": {"ma_5": 235.318, "ma_20": 245.604, "current_price": 233.78, "conservative_target": 248.1088, "aggressive_target": 256.9612, "average_price": 238.0477, "avg_volume": 2879.7123287671234, "latest_volume": 603, "volume_change_percent": -79.06040843120745, "rsi": 47.4629, "avg_gain": 0.9259, "avg_loss": 1.0249, "period": 14, "days": 90}}'

In [8]:
from decouple import config, AutoConfig
import os
os.chdir("../")
print("Current Working Directory:", os.getcwd())
print("Files in Current Directory:", os.listdir(os.getcwd()))
config = AutoConfig(search_path="/home/harry/StockTradingbot") 

DEEPSEEK_API_KEY = config("DEEPSEEK_API_KEY", default=None)

assert DEEPSEEK_API_KEY is not None, "DEEPSEEK_API_KEY is not set in the .env file"

Current Working Directory: /home/harry/StockTradingbot
Files in Current Directory: ['.env.sample', 'nbs', 'src', '.idea', '.gitignore', 'README.md', 'myenv', 'compose.yaml', 'requirements.txt', '.env', '.git']


In [ ]:
## if you have OpenAI API Key:
'''
OPENAI_API_KEY=config("OPENAI_API_KEY", default=None)
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are an expert an analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recommendation",
            "schema": {
                "type": "object",
                "properties": {
                    "buy": {
                        "description": "Recommend to buy stock",
                        "type": "boolean"
                    },
                    "sell": {
                        "description": "Recommend to sell stock",
                        "type": "boolean"
                    },
                    "hold": {
                        "description": "Recommend to hold stock",
                        "type": "boolean"
                    },
                    "explanation": {
                        "description": "Explanation of reasoning in 1 or 2 sentences",
                        "type": "string"
                    },
                    "additionalProperties": False
                }
            }
        }
    }
)

result = json.loads(response.choices[0].message.content)
result

'''


In [ ]:
## for DeepSeek Api
from openai import OpenAI

client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are an expert at analyzing stocks and respond in JSON data"},
        {"role": "user", "content": f"Considering these results {results_as_json}, provide a recommendation"}
    ],
    response_format={"type": "json_object"},  # Use "json_object" instead of "json_schema"
)

print(response.choices[0].message.content)

In [ ]:
result = json.loads(response.choices[0].message.content)
result

In [ ]:
result.get('hold') is True

In [9]:
#FOR LOGIC BASED RECOMMENDER:

In [12]:
ticker = "AAPL"
days = 90
queryset = market_services.get_daily_stock_quotes_queryset(ticker, days=days)
averages = market_services.get_daily_moving_averages(ticker, days=days, queryset=queryset)
price_target = market_services.get_price_target(ticker, days=days, queryset=queryset)
volume_trend = market_services.get_volume_trend(ticker, days=days, queryset=queryset)
rsi_data = market_services.calculate_rsi(ticker, days=days, period=14)

In [13]:
price_target

{'current_price': 233.78,
 'conservative_target': 248.1088,
 'aggressive_target': 256.9612,
 'average_price': 238.0477}

In [14]:
signals = []

In [15]:
if averages.get('ma_5') > averages.get('ma_20'):
    signals.append(1)
else:
    signals.append(-1)

In [16]:
if price_target.get('current_price') < price_target.get('conservative_target'):
    signals.append(1)
else:
    signals.append(-1)

In [17]:
volume_trend

{'avg_volume': 2879.7123287671234,
 'latest_volume': 603,
 'volume_change_percent': -79.06040843120745}

In [18]:
if volume_trend.get("volume_change_percent") > 20:
    signals.append(1)
elif volume_trend.get("volume_change_percent") < -20:
    signals.append(-1)
else:
    signals.append(0)

In [19]:
rsi_data

{'rsi': 47.4629,
 'avg_gain': 0.9259,
 'avg_loss': 1.0249,
 'period': 14,
 'days': 90}

In [20]:
rsi = rsi_data.get('rsi')
if rsi > 70:
    signals.append(-1)  # Overbought
elif rsi < 30:
    signals.append(1) # Oversold
else:
    signals.append(0)

In [21]:
score = sum(signals)
score

-1

In [22]:
if score>= 2:
    print("BUY")
elif score <= -2:
    print("SELL")
else:
    print("HOLD")

HOLD
